In [3]:
import requests
import tiktoken

from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from openai import OpenAI

from secret import OPENAI_API_KEY


### Task 1

In [8]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()["version"]["build_hash"]

'8d96bbe3bf5fed931f3119733895458eab75dca9'

In [41]:
!curl 'http://localhost:9200' --no-progress-meter | grep build_hash

    "build_hash" : "8d96bbe3bf5fed931f3119733895458eab75dca9",


In [4]:
QUERY = "How do I execute a command in a running docker container?"

### Getting the data

In [10]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework-01-intro"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-homework-01-intro'})

### Task 2

In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:03<00:00, 255.07it/s]


In [14]:
search_query_settings = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": QUERY,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
}

In [28]:
def elastic_search(search_query_settings, field: str = "_source"):
    response = es_client.search(index=index_name, body=search_query_settings)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit[field])
    
    return result_docs

### Task 3

In [16]:
elastic_search(search_query_settings, "_score")[0]

84.050095

In [18]:
search_query_settings_ml = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": QUERY,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
}

### Task #4

In [46]:
elastic_search(search_query_settings_ml, "_source")[2]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

### Task 5

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    for doc in search_results:
        context = context + context_template.format(
            question=doc['question'], text=doc['text']) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
client = OpenAI(api_key=OPENAI_API_KEY)

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [24]:
def rag(query):
    search_results = elastic_search(search_query_settings_ml)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

### Task 5

In [42]:
search_results = elastic_search(search_query_settings_ml)

In [44]:
print(build_prompt(QUERY, search_results))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [45]:
len(build_prompt(QUERY, search_results))

1462

### Task 6

In [30]:
prompt = build_prompt(QUERY, search_results)
len(tiktoken.encoding_for_model("gpt-4o").encode(prompt))

322